In [2]:
field = QQ
#  Declare the polynomial ring with indeterminate t
rationalpolynomials.<t> = QQ[] 


################################ CLASS: FiniteSet ###########################################

class FiniteSet:
    """A finite set on which we will define splits"""
    def __init__(self, setX):        
        # Define the finite set X by Python set (set) or Sage set (Set)
        self.set = setX              
        self.n = setX.cardinality()
        # Constructs R^X
        self.RX = VectorSpace(field, self.n)  
        # Construct the 1X vector. Needed to construct line segments associated to splits.
        self.Ivectors = {}   
        i = 0
        for x in sorted(self.set):
            self.Ivectors[x] = self.RX.basis()[i] 
            i = i+1                 
    def __repr__(self):
        return str(self.set)
            
# Careful! do not change self.X afterwards as the other attributes are NOT recomputed


################################  CLASS: Split ################################ 

class Split:
    """A split of X is a bipartition of X, written A|B. We also store the eventual weight here."""
                        
    def __init__(self, X, setA, weight=1):
        """Construct split with only one subset setA as input, minimizes user errors.
        Always pass a FiniteSet as X, and a Set([ ... ]) as setA."""
        if not setA.issubset(X.set):
            print("The set provided is not a subset of the chosen FiniteSet")
        elif setA.is_empty() or setA==X.set:
            print("This will not give a bipartition as one set of the pair will be empty")
        elif weight<0:
            print("Weight is non-positive.")
        else:
            # Construct the (weighted) split
            self.w = weight
            self.A = setA
            # Constructs B such that A/B is a bipartition of X. Could also use Set(X).difference(A)
            self.B = X.set - setA   
            # Could have self.S = Set([A,B]) but creating the split is not useful and won't work because unhashable sets
            # But missing out on: Set([A,B])==Set([B,A]) true   so A/B and B/A describe indeed the same split
            
            # Construct the 1_A and 1_B vectors 
            self.IA = vector(field,X.n)
            self.IB = vector(field,X.n)
            for x in self.A:
                self.IA += X.Ivectors[x]
            for x in self.B:
                self.IB += X.Ivectors[x]
            # Construct the vector p_sigma associated to a split sigma
            self.pvector = self.B.cardinality()/X.n * self.IA - self.A.cardinality()/X.n * self.IB 
            # Construct the line segment S_sigma associated to a split sigma
            self.line_segment = Polyhedron(vertices = [self.pvector, -self.pvector])
            
    def __repr__(self):
        """Display the split in A|B format"""
        displayA = "{" + str(', '.join(sorted(self.A))) + "}"
        displayB = "{" + str(', '.join(sorted(self.B))) + "}"
        return displayA + " | " + displayB +  " weight " + str(self.w)
    
    def kronecker(self, x, y):
        """Returns the value of the delta function associated to the given split, 
        for the elements x, y of the parent set."""
        if (x in self.A and y in self.A) or (x in self.B and y in self.B): 
            return 0
        else:
            return 1            
                                         
    def is_trivial(self): 
        "Checks if a split is trivial or not"
        return (len(self.A)==1 or len(self.B)==1)
    
    
################################################################
        
def are_compatible(S1, S2):
    "Checks if two splits are compatible"
    cond1 = S1.A.intersection(S2.A).is_empty()
    cond2 = S1.A.intersection(S2.B).is_empty()
    cond3 = S1.B.intersection(S2.A).is_empty()
    cond4 = S1.B.intersection(S2.B).is_empty()
    return cond1 or cond2 or cond3 or cond4 
        

    
################################  CLASS: SplitSystem #################################

class SplitSystem:
    """(Weighted) split system : A pair (S,alpha) where S is a syst of splits on X and alpha is a weighting.\
    A system of splits on X is simply a set of splits on X"""
    def __init__(self, X):
        self.X = X
        # List of Splits that form the split system
        # Use this later with S.splits.extend([S1,S2,S3,S4,S5]), where S1,...,S5 are instances of Split
        self.splits = []               
        
    def __repr__(self):
        nice = ""
        for x in self.splits:
            if x.w>0:
                nice += str(x) + " ;  " # + "\n"
        nice = nice[:-4] # To remove last ";"
        return nice
    
    def d_alpha(self, x, y):
        """Returns the value of the split-decomposable pseudometric associated to the given split system"""
        return sum(self.splits[n].w * self.splits[n].kronecker(x,y) for n in range(len(self.splits)))
    
    def is_compatible(self): # Must only compare those with non-zero weight
        for S1 in self.splits:                      
            if S1.w>0:
                for S2 in self.splits:
                    if S2.w>0:
                        if not are_compatible(S1,S2):
                            return False
        return True
    
    def is_metric(self):
        for x in self.X.set:
            for y in self.X.set:
                if x != y and self.d_alpha(x,y) == 0:
                    return False
        return True
        
    
    def zonotope(self):
        """Construct the zonotope Z(S) associated to a split system S"""
        Z_list = []
        for sigma in self.splits:
            if sigma.w>0: # When we also consider splits with weight 0, don't count them
                Z_list.append(sigma.line_segment)
        # Special case for when the split system is empty, then zonotope is empty polyhedron
        # (for exemple generic metric type II on 5 points with all parameters 0 except alpha)
        if not Z_list:
            Z = Polyhedron(base_ring = QQ, ambient_dim = self.X.n)
        # General case, zonotope is the Minkowski sum of the line segments
        else:
            Z = sum(Z_list)  # Calls the overriden + for polyhedra, which is Minkowski sums
        return Z
    
    def zonotope_generating_vectors(self):
        """Returns the list of p and -p vectors of each split, which generate the zonotope of the split system.
        To be used to call erhart_poly function."""
        gv_list =[]
        for sigma in self.splits:
            if sigma.w>0:
                gv_list.append(sigma.pvector)
                gv_list.append(-sigma.pvector)
        return gv_list
        
    

################################  FUNCTION : Ehrhart polynomial ################################

def ehrhart_poly(list_of_vectors, setX): # Pass a list of vectors like [vector([1,0,0]),vector([1,0,0])] (or RX([1,0,0]))
    """Calculate Ehrhart polynomial provided list of vectors generating the zonotope. 
    Book: Computing the continuous discretely p177."""
    #  Must translate the list into a dictionnary because "error : mutable vectors are unhashable", 
    #  so we could not compute subsets of the list otherwise.
    #  Want to obtain something of the form vectorlist = {0:vector([0,4]), 1:vector([3,3]), 2:vector([4,1])}
    vectorlist = {}
    i = 0
    for entry in list_of_vectors:
        vectorlist[i] = entry
        i = i + 1
    # If the list is empty, it corresponds to the empty zonotope, the Ehrhart polynomial is 0, 
    # there are no integer points
    if vectorlist == {}:
        return t*0
    else:
        # Construct the list of subsets of the given list of vectors. 
        # The trick is to use Subset() on the keys of vectorlist, which are hashable (integers).
        subsetslist = []          
        for k in Subsets(range(len(vectorlist))):
            subsetslistitem = []
            for kk in k:                                 #  Because k is a set, not an iterator
                subsetslistitem.append(vectorlist[kk])   #  Fortunately empty set is also present
            subsetslist.append(subsetslistitem)
        # Of these subsets, contruct the list of linearly independant ones. 
        lin_indep_subsets = []
        for sub in subsetslist:
            if not setX.RX.are_linearly_dependent(sub): # (provide a list of vectors to this function)
                lin_indep_subsets.append(sub)    

        # Compute the polynomial: is a sum on S where S ranges over all linearly independant subsets
        polynomial = 0 
        for S in lin_indep_subsets:
            size = len(S)
            M = matrix.column(field, S)     # Matrix whose columns are the elements of S
            allminors = M.minors(size)      # Find GCD of all minors of size |S| 
            mS = gcd(allminors)
            polynomial += mS * t^size

        return polynomial
    
################################  FUNCTION : results ################################
    
def results(metric):
    """Compute the needed results for a split system S"""
    X = metric.X
    n = X.n
    ### Number of splits
    number = 0
    for s in metric.S.splits:
        if s.w > 0 :# If weight of split positive
            number += 1
    
    ### Compute Fundamental and Lipschitz polytopes
    if n==4:
        Lip = lipschitz_polytope_4pointsset(metric)
        if metric.S.is_metric():
            Fund = fundamental_polytope_4pointsset(metric)
            fvec_Fund = Fund.f_vector()
            intpts_Fund = len(Fund.integral_points())
        else:
            # Fundamental polytope is not defined if not a metric
            no = "not a metric"
            Fund, fvec_Fund = no, no
            intpts_Fund = "-"
            
    elif n==5:
        Lip = lipschitz_polytope_5pointsset(metric)
        Fund = fundamental_polytope_5pointsset(metric)
        fvec_Fund = Fund.f_vector()
        intpts_Fund = len(Fund.integral_points())
              
    ### Compute Zonotope from Minkowski sum
    Z = metric.S.zonotope()
    # Scale zonotope to make it rational and calculate Ehrhart polynomial and number of integer points
    gen_vec = metric.S.zonotope_generating_vectors()
    Zscaled = X.n * Z
    gen_vec_scaled = []
    for vec in gen_vec:
        gen_vec_scaled.append(X.n * vec)
    h(t) = ehrhart_poly(gen_vec_scaled, X)
    intptsscaled = h(1)
    
    return [metric.S, number, metric.S.is_compatible(), Lip == Z,
            fvec_Fund, intpts_Fund,
            Lip.f_vector(), len(Lip.integral_points()), Z.f_vector(), len(Z.integral_points()), h(t), intptsscaled]

header = ["Split system", "Number of splits", "Compatible sys.", "Lip.polyt.=Zonotope", 
          "f-vector Fund.p.", "Int. pts in Fund.",
          "f-vector Lip.p.", "Int. pts in Lip.", "f-vector of Z", "Int. pts in Z", # Add quasi polynomial?
          "Ehrhart polynomial of Zscaled", "Int. pts in Zscaled"] 
    
